# Pre-processing - Get synchronization times

This notebook create a file with times of the stimuli for a particular record.

In [ ]:
# import sys
# sys.path.append('../src')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mealib.utils import check_directory
from mealib.preprocessing import Sync

%matplotlib notebook 

In [1]:
experiment_name = 'MR-0242'
# experiment_name = '2016-06-17'

mcd_file = '../data/raw_data/'+experiment_name+'/'+experiment_name+'_analog.mcd'
# mcd_file = '../data/raw_data/chirp_droga/'+experiment_name+'_analog.mcd'
output_folder = '../data/sync/'+experiment_name+'/'
check_directory(output_folder)

real_fps = 59.7694

# Get synchronization signal
## Matlab script

## python function

In [2]:
sync_data = Sync(experiment_name,real_fps)

sync_data.read_mcd(mcd_file)
sync_data.show_entities()

Entities in MCD file.
index:  	 label:  	 entity_type: 
0000: anlg0001 0254 0000       A3 type: 2


In [3]:
sync_data.data.metadata_raw

{'AppName': 'MC_Rack 4.6.2.0',
 'EntityCount': 1,
 'FileComment': '',
 'FileType': 'Conf cont An',
 'TimeSpan': 6409.1,
 'TimeStampResolution': 5e-05,
 'Time_Day': 18,
 'Time_Hour': 17,
 'Time_MilliSec': 894,
 'Time_Min': 0,
 'Time_Month': 4,
 'Time_Sec': 25,
 'Time_Year': 2018}

In [4]:
mcd_channel = 0

sync_data.analyzer(mcd_channel)

MemoryError: 

## Read start-end file and create event list
From sync file created above we compute the difference between all frames and we only take the frames with a difference more than one frame to create a event.

In [ ]:
sync_data.create_events()
sync_data.event_list.head()

### Add repetition frame time  in csv file

In [ ]:
sync_data.add_repeated()
sync_data.event_list.head()

In [ ]:
event_list = sync_data.event_list
filter_frame = event_list['n_frames'] > -1
event_list[filter_frame]

### Check repeated


In [ ]:
mcd_channel = 0
start_point = 34877499-1000
window = 3000
sync_data.plot_window( mcd_channel, start_point, window )

## Save csv file

In [ ]:
sync_data.save_analyzed(output_folder)

In [ ]:
sync_data.save_events(output_folder)

In [ ]:
sync_data.close_file()

**Note:** The times in event_duration have small differences with the theoric times, it's because the theorical FPS of the projector is different to the real FPS.   

For example, projector show theoricly 60 FPS but the real FPS is 59.7523. Please ckeck it in the log file.
* 2100 images to 60 [fps] = 35 [s]
* 2100 images to 59.7523 [fps] = 35.145 [s]
The differences could be significant!

## Create separated sync files for each event


In [ ]:
experiment_name = 'MR-0242'

source_folder = '../data/sync/'+experiment_name+'/'
output_folder = '../data/sync/'+experiment_name+'/event_list/'
check_directory(output_folder)

sync_reload = Sync(experiment_name)
sync_reload.load_analyzed(source_folder)
sync_reload.load_events(source_folder + 'event_list_' + sync_reload.exp_name + '_.csv')
sync_reload.create_separated_sync(output_folder)


In [ ]:
# exp = {'MR-0117' : 0,'MR-0118' : 0,'MR-0119' : 0,'MR-0120' : 0,'MR-0121' : 0,'MR-0122' : 0,'MR-0123' : 0,'MR-0124' : 0,
#        'MR-0125' : 0,'MR-0126' : 0,'MR-0127' : 0,'MR-0128' : 0,'MR-0129' : 0,'MR-0130' : 0,'MR-0131' : 0,'MR-0133' : 0,
#        'MR-0135' : 0,'MR-0145' : 0,'MR-0146' : 0,'MR-0147' : 0,'MR-0148' : 0,'MR-0149' : 0,'MR-0174' : 0,'MR-0227' : 0,
#        'MR-0228':1,'MR-0242':0}
exp = {'MR-0261':0,'MR-0261-2':1,'MR-0262':0,'MR-0263':0,
        'MR-0263-2':0,'MR-0264':0}

for kexp in exp:
    source_folder = '../data/sync/'+kexp+'/'
    print('\n'+kexp)
    sync_data = Sync(kexp,real_fps)
    sync_data.read_mcd('../data/raw_data/'+kexp+'/'+kexp+'_analog.mcd')
    sync_data.show_entities()
    sync_data.plot_window(2,5*60*20000,20000)

In [ ]:
start_point = 10000360-335
duration_points = 335*5
# for kchannel in range(252,256):
for kchannel in range(2):
    sync_data.plot_window(kchannel,start_point,duration_points)

In [ ]:
def getTimes(start_point, duration_points = 335*7):
    ss,nada,nada = sync_data.data.entities[254].get_data(start_point,duration_points)
    ss = ss/np.amax(ss)
    ld,nada,nada = sync_data.data.entities[253].get_data(start_point,duration_points)
    ld = ld/np.amax(ld)
    plt.plot(ss,alpha=0.9,label='Sync signal',lw=0.3)
    plt.plot(ld,alpha=0.9,label='Photodiode',lw=0.3)
    plt.xlabel('Time [samples]')
    plt.ylabel('Normalized amplituded [V]')
    plt.legend()


plt.figure(figsize=(10,5))
plt.subplot(1,2,1).set_title('Start of a sequence of imgenes')
getTimes(start_point = 154360-335*2)
plt.subplot(1,2,2).set_title('End of a sequence of imgenes')
getTimes(start_point = 857923-335*5)
# plt.savefig('../figures/Photodiode_record.png')

In [ ]:
import pyMCStream as mc
pd_data = b'/home/cesar/exp/MEA-analysis/data/raw_data/test/test_analog.mcd'
data = mc.open_stream(pd_data,b'anlg0001')



In [ ]:
data.shape

In [ ]:
data, time, dur = sync_data.get_raw_data(131)

In [ ]:
data.shape
plt.figure()
plt.plot(data[154360:857923])